In [25]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
# ignore warnings
import warnings
warnings.filterwarnings("ignore")


plays = pd.read_csv('plays.csv')
players = pd.read_csv('players.csv')
week1 = pd.read_csv('week1.csv')
#plays = pd.read_csv('plays.csv')
#plays = pd.read_csv('plays.csv')

plays.head()

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,playType,yardlineSide,yardlineNumber,offenseFormation,personnelO,defendersInTheBox,numberOfPassRushers,personnelD,typeDropback,preSnapVisitorScore,preSnapHomeScore,gameClock,absoluteYardlineNumber,penaltyCodes,penaltyJerseyNumbers,passResult,offensePlayResult,playResult,epa,isDefensivePI
0,2018090600,75,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,ATL,play_type_pass,ATL,20,I_FORM,"2 RB, 1 TE, 2 WR",7.0,4.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0.0,0.0,15:00:00,90.0,NaN,NaN,C,10,10,0.261827,False
1,2018090600,146,(13:10) M.Ryan pass incomplete short right to ...,1,1,10,ATL,play_type_pass,PHI,39,SINGLEBACK,"1 RB, 1 TE, 3 WR",7.0,4.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0.0,0.0,13:10:00,49.0,NaN,NaN,I,0,0,-0.372360,False
2,2018090600,168,(13:05) (Shotgun) M.Ryan pass incomplete short...,1,2,10,ATL,play_type_pass,PHI,39,SHOTGUN,"2 RB, 1 TE, 2 WR",6.0,4.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0.0,0.0,13:05:00,49.0,NaN,NaN,I,0,0,-0.702779,False
3,2018090600,190,(13:01) (Shotgun) M.Ryan pass deep left to J.J...,1,3,10,ATL,play_type_pass,PHI,39,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,5.0,"4 DL, 1 LB, 6 DB",SCRAMBLE_ROLLOUT_LEFT,0.0,0.0,13:01:00,49.0,NaN,NaN,C,33,33,3.047530,False
4,2018090600,256,(10:59) (Shotgun) M.Ryan pass incomplete short...,1,3,1,ATL,play_type_pass,PHI,1,SHOTGUN,"2 RB, 3 TE, 0 WR",8.0,6.0,"6 DL, 3 LB, 2 DB",TRADITIONAL,0.0,0.0,10:59:00,11.0,NaN,NaN,I,0,0,-0.842272,False


In [2]:
pd.set_option('display.max_columns',None)
plays

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,playType,yardlineSide,yardlineNumber,offenseFormation,personnelO,defendersInTheBox,numberOfPassRushers,personnelD,typeDropback,preSnapVisitorScore,preSnapHomeScore,gameClock,absoluteYardlineNumber,penaltyCodes,penaltyJerseyNumbers,passResult,offensePlayResult,playResult,epa,isDefensivePI
0,2018090600,75,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,ATL,play_type_pass,ATL,20,I_FORM,"2 RB, 1 TE, 2 WR",7.0,4.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0.0,0.0,15:00:00,90.0,NaN,NaN,C,10,10,0.261827,False
1,2018090600,146,(13:10) M.Ryan pass incomplete short right to ...,1,1,10,ATL,play_type_pass,PHI,39,SINGLEBACK,"1 RB, 1 TE, 3 WR",7.0,4.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0.0,0.0,13:10:00,49.0,NaN,NaN,I,0,0,-0.372360,False
2,2018090600,168,(13:05) (Shotgun) M.Ryan pass incomplete short...,1,2,10,ATL,play_type_pass,PHI,39,SHOTGUN,"2 RB, 1 TE, 2 WR",6.0,4.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0.0,0.0,13:05:00,49.0,NaN,NaN,I,0,0,-0.702779,False
3,2018090600,190,(13:01) (Shotgun) M.Ryan pass deep left to J.J...,1,3,10,ATL,play_type_pass,PHI,39,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,5.0,"4 DL, 1 LB, 6 DB",SCRAMBLE_ROLLOUT_LEFT,0.0,0.0,13:01:00,49.0,NaN,NaN,C,33,33,3.047530,False
4,2018090600,256,(10:59) (Shotgun) M.Ryan pass incomplete short...,1,3,1,ATL,play_type_pass,PHI,1,SHOTGUN,"2 RB, 3 TE, 0 WR",8.0,6.0,"6 DL, 3 LB, 2 DB",TRADITIONAL,0.0,0.0,10:59:00,11.0,NaN,NaN,I,0,0,-0.842272,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,2018122200,2300,(7:53) J.Johnson pass incomplete short left [K...,3,2,5,WAS,play_type_unknown,WAS,31,SINGLEBACK,"1 RB, 1 TE, 3 WR",7.0,NaN,"2 DL, 4 LB, 5 DB",NaN,NaN,NaN,NaN,NaN,DH,TEN 24,I,0,5,0.848703,False
19235,2018122200,3177,(6:53) (Shotgun) B.Gabbert pass incomplete sho...,4,3,7,TEN,play_type_unknown,WAS,37,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,NaN,"2 DL, 4 LB, 5 DB",NaN,NaN,NaN,NaN,NaN,DH,WAS 31,I,0,5,1.227695,False
19236,2018122201,566,(5:32) (Shotgun) P.Rivers pass deep right to K...,1,3,4,LAC,play_type_unknown,LAC,49,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,NaN,"1 DL, 5 LB, 5 DB",NaN,NaN,NaN,NaN,NaN,OPI,LAC 13,C,27,-10,-0.720168,False
19237,2018122201,1719,(1:08) P.Rivers pass incomplete deep middle to...,2,3,1,LAC,play_type_unknown,LAC,48,SINGLEBACK,"1 RB, 1 TE, 3 WR",7.0,NaN,"2 DL, 3 LB, 6 DB",NaN,NaN,NaN,NaN,NaN,DPI,BAL 23,I,0,15,1.336078,True


In [3]:
plays.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   gameId                  19239 non-null  int64  
 1   playId                  19239 non-null  int64  
 2   playDescription         19239 non-null  object 
 3   quarter                 19239 non-null  int64  
 4   down                    19239 non-null  int64  
 5   yardsToGo               19239 non-null  int64  
 6   possessionTeam          19239 non-null  object 
 7   playType                19239 non-null  object 
 8   yardlineSide            18985 non-null  object 
 9   yardlineNumber          19239 non-null  int64  
 10  offenseFormation        19098 non-null  object 
 11  personnelO              19210 non-null  object 
 12  defendersInTheBox       19177 non-null  float64
 13  numberOfPassRushers     18606 non-null  float64
 14  personnelD              19210 non-null

In [4]:
#plays["offenseFormation"].replace({"I_FORM": "IFORM"}, inplace=True)

#plays = plays[plays['offenseFormation'].notna()]

#plays = plays[plays['typeDropback'].notna()]

#plays.personnelD.isna().sum()



#ohe = OneHotEncoder(sparse=False)

#ohe.fit_transform(plays[['personnelO']]).shape

#plays['possessionTeam'].replace({'TRADITIONAL':0,'SCRAMBLE_ROLLOUT_RIGHT':1,
#                                 'SCRAMBLE':2,'DESIGNED_ROLLOUT_RIGHT':3,
#                                 'SCRAMBLE_ROLLOUT_LEFT':4,'DESIGNED_ROLLOUT_LEFT':5,
#                                 'UNKNOWN':6}, inplace=True)

#plays['offenseFormation'].replace({'SHOTGUN':0,'SINGLEBACK':1,'EMPTY':2,'I_FORM':3,
#                                   'PISTOL':4,'JUMBO':5,'WILDCAT':6}, inplace=True)

#plays['possessionTeam'].replace({'PIT':0,'GB':1,'TB':2,'IND':3,'ATL':4,'PHI':5,'NYG':6,'DET':7,
#                                'CLE':8,'MIN':9,'LA':10,'OAK':11,'NE':12,'BAL':13,'CAR':14,
#                                'DAL':15,'CIN':16,'DEN':17,'JAX':18,'KC':19,'HOU':20,'WAS':21,
#                                'SF':22,'NYJ':23,'ARI':24,'NO':25,'BUF':26,'CHI':27,'MIA':28,
#                                'LAC':29,'TEN':30,'SEA':31}, inplace=True)

In [5]:
def prep_plays_data(df):
    #df = acquire_plays_data.get_plays_data()
    
    df = df[['playDescription', 'quarter', 'down', 'yardsToGo', 'possessionTeam',
             'offenseFormation', 'personnelO', 'defendersInTheBox', 'numberOfPassRushers', 
             'personnelD', 'typeDropback', 'gameClock', 'absoluteYardlineNumber', 'epa', 'playType','passResult']]
    
    #df.personnelO = df.personnelO.str.replace(r'\D',"")
    #df.personnelD = df.personnelD.str.replace(r'\D',"")
    
    df = df[df.playType == 'play_type_pass']
    
    df["passResult"].replace({"I": "0", "C": "1", "IN":"0"}, inplace=True)
    
    df['passResult'] = pd.to_numeric(df['passResult'])
    
    df = df[df.passResult != 'R']
    
    df = df[df['offenseFormation'].notna()]
    df = df[df['typeDropback'].notna()]
    
    return df

In [6]:
plays = prep_plays_data(plays)
plays.head()

,playDescription,quarter,down,yardsToGo,possessionTeam,offenseFormation,personnelO,defendersInTheBox,numberOfPassRushers,personnelD,typeDropback,gameClock,absoluteYardlineNumber,epa,playType,passResult
0,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,ATL,I_FORM,"2 RB, 1 TE, 2 WR",7.0,4.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,15:00:00,90.0,0.261827,play_type_pass,1
1,(13:10) M.Ryan pass incomplete short right to ...,1,1,10,ATL,SINGLEBACK,"1 RB, 1 TE, 3 WR",7.0,4.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,13:10:00,49.0,-0.372360,play_type_pass,0
2,(13:05) (Shotgun) M.Ryan pass incomplete short...,1,2,10,ATL,SHOTGUN,"2 RB, 1 TE, 2 WR",6.0,4.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,13:05:00,49.0,-0.702779,play_type_pass,0
3,(13:01) (Shotgun) M.Ryan pass deep left to J.J...,1,3,10,ATL,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,5.0,"4 DL, 1 LB, 6 DB",SCRAMBLE_ROLLOUT_LEFT,13:01:00,49.0,3.047530,play_type_pass,1
4,(10:59) (Shotgun) M.Ryan pass incomplete short...,1,3,1,ATL,SHOTGUN,"2 RB, 3 TE, 0 WR",8.0,6.0,"6 DL, 3 LB, 2 DB",TRADITIONAL,10:59:00,11.0,-0.842272,play_type_pass,0


In [7]:
df = plays[['quarter','down','possessionTeam',
            'offenseFormation','defendersInTheBox',
            'numberOfPassRushers','typeDropback']]

In [17]:
X = plays[['quarter','down','possessionTeam',
            'offenseFormation','defendersInTheBox',
            'numberOfPassRushers','typeDropback']]
y = plays.passResult

In [34]:
X_train_validate, X_test, y_train_validate, y_test = train_test_split(X, y, test_size = .20, random_state = 123)

X_train, X_validate, y_train, y_validate = train_test_split(X_train_validate, y_train_validate, test_size = .30, random_state = 123)

print("train: ", X_train.shape, ", validate: ", X_validate.shape, ", test: ", X_test.shape)
print("train: ", y_train.shape, ", validate: ", y_validate.shape, ", test: ", y_test.shape)

train:  (9657, 7) , validate:  (4140, 7) , test:  (3450, 7)
train:  (9657,) , validate:  (4140,) , test:  (3450,)


In [46]:
column_trans = make_column_transformer((OneHotEncoder(),
                                        ['offenseFormation','typeDropback','possessionTeam']),
                                      remainder='passthrough')
logreg = LogisticRegression(solver='lbfgs')
decisiontree = tree.DecisionTreeClassifier(max_depth=5)
svm = SVC
naivebayes = GaussianNB

In [48]:
pipe = make_pipeline(column_trans, logreg)
pipe_dt = make_pipeline(column_trans, decisiontree)
pipe_svm = make_pipeline(column_trans, svm)
pipe_nb = make_pipeline(column_trans, naivebayes)

In [63]:
print('Logistic Regression with 7 features')
print('Accuracy:',cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy').mean())
print('Precision:',cross_val_score(pipe, X_train, y_train, cv=5, scoring='precision').mean())
print('Recall:',cross_val_score(pipe, X_train, y_train, cv=5, scoring='recall').mean())
print('F1 Score:',cross_val_score(pipe, X_train, y_train, cv=5, scoring='f1').mean())

Logistic Regression with 7 features
Accuracy: 0.6572431602501627
Precision: 0.66649565465522
Recall: 0.955308468337716
F1 Score: 0.7851612750344715


In [67]:
#print('Decision Tree with 7 features')
#print('Accuracy:',cross_val_score(estimator = clf, pipe_dt, X_train, y_train, cv=5, n_jobs=4,scoring='accuracy').mean())
#print('Precision:',cross_val_score(pipe_nb, X_train, y_train, cv=5, scoring='precision').mean())
#print('Recall:',cross_val_score(pipe_nb, X_train, y_train, cv=5, scoring='recall').mean())

In [53]:
#models = [GaussianNB(), DecisionTreeClassifier(), SVC(), LogisticRegression(solver='lbfgs')]
#names = ["Naive Bayes", "Decision Tree", "SVM","Logistic Regression"]
#for model, name in zip(models, names):
#    print(name)
#    for score in ["accuracy", "precision", "recall"]:
#        print(score)
#        print (cross_val_score(pipe, X_train, y_train, cv=5, scoring=score).mean())

In [40]:
logit = LogisticRegression(C=1, class_weight={0:1, 1:99}, random_state=123, intercept_scaling=1, solver='lbfgs')

In [54]:
#y_pred = logit.predict(pipe, X_train)

In [13]:
X = df[['quarter','down','yardsToGo']]
y = df[['passResult']]

In [14]:
X_train_validate, X_test, y_train_validate, y_test = train_test_split(X, y, test_size = .20, random_state = 123)

X_train, X_validate, y_train, y_validate = train_test_split(X_train_validate, y_train_validate, test_size = .30, random_state = 123)

print("train: ", X_train.shape, ", validate: ", X_validate.shape, ", test: ", X_test.shape)
print("train: ", y_train.shape, ", validate: ", y_validate.shape, ", test: ", y_test.shape)

train:  (9713, 3) , validate:  (4163, 3) , test:  (3470, 3)
train:  (9713, 1) , validate:  (4163, 1) , test:  (3470, 1)


In [15]:
# from sklearn.linear_model import LogisticRegression
logit = LogisticRegression(C=1, class_weight={0:1, 1:99}, random_state=123, intercept_scaling=1, solver='lbfgs')

In [68]:
#logit.fit(X_train, y_train)

In [ ]:
LogisticRegression(C=1, class_weight={0: 1, 1: 99}, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=123, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
y_pred = logit.predict(X_train)

In [ ]:
y_pred_proba = logit.predict_proba(X_train)

In [ ]:
print('Accuracy of Logistic Regression classifier on training set: {:.2f}'
     .format(logit.score(X_train, y_train)))

In [ ]:
print(confusion_matrix(y_train, y_pred))

In [ ]:
print(classification_report(y_train, y_pred))